# Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import config
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from tqdm import tqdm
from collections import defaultdict
import warnings

# Load data

In [2]:
X = pd.read_csv(os.path.join(config.CLEAN_DIR, "taxonomic_features.csv"), index_col=[0, 1])
y = pd.read_csv(os.path.join(config.CLEAN_DIR, "metadata.csv"), index_col=[0, 1]).iloc[:, [0]]
display(X.head())
display(y.head())

k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID Sample Accession                                            
GMHI-10  SAMN03283239        0.000000                     0.000000   
         SAMN03283266        0.000000                     0.000000   
         SAMN03283281        0.009764                     0.009764   
         SAMN03283245        0.106067                     0.106067   
         SAMN03283294        0.000000                     0.000000   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID Sample Accession                                                   
GMHI-10  SAMN03283239                                            0.000000   
         SAMN03283266                                            0.000000   
         SAMN03283281                                            0.009764   
         SAMN03283245                                            0.106067   
         SAMN03283294                                            0.000000   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID Sample Accession                                                                         
GMHI-10  SAMN03283239                                               0.000000                      
         SAMN03283266                                               0.000000                      
         SAMN03283281                                               0.009764                      
         SAMN03283245                                               0.106067                      
         SAMN03283294                                               0.000000                      

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae  \
Study_ID Sample Accession                                                                                                
GMHI-10  SAMN03283239                                               0.000000                                             
         SAMN03283266                                               0.000000                                             
         SAMN03283281                                               0.009764                                             
         SAMN03283245                                               0.106067                                             
         SAMN03283294                                               0.000000                                             

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter  \
Study_ID Sample Accession                                                                                                                      
GMHI-10  SAMN03283239                                               0.000000                                                                   
         SAMN03283266                                               0.000000                                                                   
         SAMN03283281                                               0.009764                                                                   
         SAMN03283245                                               0.072453                                                                   
         SAMN03283294                                               0.000000                                                                   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
GMHI-10  SAMN03283239                                               0.000000              

PHENOTYPE:Healthy_Nonhealthy
Study_ID Sample Accession                              
GMHI-10  SAMN03283239                              True
         SAMN03283266                              True
         SAMN03283281                              True
         SAMN03283245                              True
         SAMN03283294                              True

# Generate group splits

In [3]:
# indentify groups to partition dataset

sample_studies = np.array(X.index.get_level_values(0))
o = OrdinalEncoder()
groups = o.fit_transform(sample_studies.reshape((len(sample_studies), 1))).flatten()
logo = LeaveOneGroupOut()

# Iterate through all regularization and cutoff values

In [4]:
def get_ISV_scores(reg):
    # instantiate the logistic regression classifier (gmhi2)
    gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=reg, class_weight="balanced")

    # generate logit predictions for each left out study
    GMHI2_scores_ISV = cross_val_predict(gmhi2, X > config.PRESENCE_CUTOFF, y.values.flatten(), method="decision_function",
        groups=groups, cv=logo, verbose=2, n_jobs=-1
    )
    GMHI2_scores_ISV = pd.DataFrame(GMHI2_scores_ISV, index=y.index, columns=["GMHI2_ISV"])
    return GMHI2_scores_ISV

In [5]:
def get_training_set_scores(reg):
    # instantiate the logistic regression classifier (gmhi2)
    gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=reg, class_weight="balanced")
    
    # train and test on training set
    gmhi2.fit(X > config.PRESENCE_CUTOFF, y.values.flatten())
    GMHI2_scores_training_set = gmhi2.decision_function(X > config.PRESENCE_CUTOFF)
    GMHI2_scores_training_set = pd.DataFrame(GMHI2_scores_training_set, index=y.index, columns=["GMHI2_training_set"])
    return GMHI2_scores_training_set

In [6]:
def get_mean_study_bal(labels, scores):
    bals = []
    for study in np.unique(labels.index.get_level_values(0)):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            score = balanced_accuracy_score(labels.loc[study], scores.loc[study] > 0)
        bals.append(score)
    return np.mean(bals)

In [7]:
def get_info(scores, reg):
    cutoffs = np.arange(0, float(abs(scores).max()), 0.1)
    bals = []
    accs = []
    mean_study_bals = []
    num_retaineds = []
    for cutoff in cutoffs:
        idx = (abs(scores) >= cutoff).values
        curr_scores = scores.iloc[idx]
        curr_labels = y.iloc[idx]
        bal = balanced_accuracy_score(curr_labels, curr_scores > 0)
        acc = accuracy_score(curr_labels, curr_scores > 0)
        mean_study_bal = get_mean_study_bal(curr_labels, curr_scores)
        num_retained = curr_scores.shape[0]
        bals.append({"C" : reg, "cutoff" : cutoff, "score" : bal})
        accs.append({"C" : reg, "cutoff" : cutoff, "score" : acc})
        mean_study_bals.append({"C" : reg, "cutoff" : cutoff, "score" : mean_study_bal})
        num_retaineds.append({"C" : reg, "cutoff" : cutoff, "score" : num_retained})
    return bals, accs, mean_study_bals, num_retaineds

In [8]:
ISV = defaultdict(list)
training_set = defaultdict(list)


for reg in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 1, 2, 3]:
    
    #-------------------------------ISV--------------------------
    ISV_scores = get_ISV_scores(reg)
    bals, accs, mean_study_bals, num_retaineds = get_info(ISV_scores, reg)
    ISV["bals"] += bals
    ISV["accs"] += accs
    ISV["mean_study_bals"] += mean_study_bals
    ISV["num_retaineds"] += num_retaineds
    #-------------------------------ISV--------------------------
    
    
    
    #-------------------------------Training set--------------------------
    training_set_scores = get_training_set_scores(reg)
    bals, accs, mean_study_bals, num_retaineds = get_info(training_set_scores, reg)
    training_set["bals"] += bals
    training_set["accs"] += accs
    training_set["mean_study_bals"] += mean_study_bals
    training_set["num_retaineds"] += num_retaineds
    #-------------------------------Training set--------------------------

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done  55 out of  57 | elapsed:   15.7s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  57 out of  57 | elapsed:   16.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  55 out of  57 | elapsed:   11.5s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  57 out of  57 | elapsed:   11.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  55 out of  57 | elapsed:   11.7s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  57 out of  57 | elapsed:   11.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elaps

In [9]:
for key in ISV:
    df = pd.DataFrame(ISV[key])
    df = df.set_index("cutoff", drop=True)
    df = df.pivot(columns="C")
    df.index = df.index.rename("")
    df.columns = df.columns.get_level_values(1)
    df.columns = [f"C={col}" for col in df.columns]
    name = f"ISV_hyperparameter_tuning_{key}.csv"
    df.to_csv(os.path.join(config.LOG_DIR, name))

In [10]:
for key in training_set:
    df = pd.DataFrame(training_set[key])
    df = df.set_index("cutoff", drop=True)
    df = df.pivot(columns="C")
    df.index = df.index.rename("")
    df.columns = df.columns.get_level_values(1)
    df.columns = [f"C={col}" for col in df.columns]
    name = f"training_set_hyperparameter_tuning_{key}.csv"
    df.to_csv(os.path.join(config.LOG_DIR, name))